In [157]:
import math
import numpy
import pandas as pd

## General Properties

In [158]:
mechanical_eff = 0.99
# Air Properties
gamma_air = 1.4
gamma_g = 1.33333
c_p_air = 1.005
c_p_gas = 1.148
R = 287

## Define Ranges

Define the range for the stage loading coefficient, flow coefficient and the reaction. Using the obtained values, calculate the other values.

Use the following data:
1. Stage Loading Coefficient -> 2.5 - 5.0
2. Reaction -> 0.45 - 0.49
3. Flow Coefficient -> TBD

In [159]:
range_controller = 50
range_flow_coefficient = numpy.linspace(0.3, 1.0, range_controller)
range_degree_reaction = numpy.linspace(0.4, 0.5, range_controller)
range_alpha_3 = numpy.linspace(10, 40, range_controller)
range_exit_axial_velocity = numpy.linspace(0.3 * numpy.sqrt(gamma_g*R*1020.985), 0.55 * numpy.sqrt(gamma_g*R*1020.985), range_controller) # hyp
#====================================================================#
c_a_exit = 255                  # [m/s]
U_max = 335                     # [m/s]

### CHECK FILTER
1. Remove the values of U based on the maximum hub speed.

In [160]:
# Remove the values of U based on the maximum hub speed.
# CALCULATION OF METAL SPEED U
U = c_a_exit/range_flow_coefficient
upper_limit_U = 335
indices = numpy.where(U <= upper_limit_U)[0]
U = U[indices]

2. Remove the values of the stage loading based on industry practices.

In [161]:
# CALCULATION OF THE STAGE LOADING COEFFICIENT
# Filter the values for stage loading
range_flow_coefficient = range_flow_coefficient[indices]
upper_limit_stage_loading = 3.0*2
stage_loading_we_get = 2*216000/(U**2)
indices = numpy.where(stage_loading_we_get <= upper_limit_stage_loading)[0]
stage_loading_we_get = stage_loading_we_get[indices]
range_flow_coefficient = range_flow_coefficient[indices]

## Computation of results
Use of dataframe to organize and analyze the data. 

Results saved as .csv file -> "_outputs/output.csv"

In [162]:
def calc_exit_velocity(alpha):
    #beta_3 = numpy.arctan(numpy.tan(numpy.radians(alpha)) + range_flow_coefficient**(-1))
    velocity = c_a_exit/(numpy.cos(alpha))
    velocity_mach = velocity/numpy.sqrt(gamma_g*R*1020.985)
    return velocity_mach


def calc_reaction(alpha, range_stage_loading, range_flow_coefficient):
    """
    This function calculates the degree of reaction.
    Input: Swirl angle "alpha", flow coefficient "phi", blade loading coefficient "psi"
    Output: Returns the value of the degree of coefficient.
    """
    tan_beta_3 = numpy.tan(numpy.radians(alpha)) + range_flow_coefficient**(-1)
    reaction = (tan_beta_3 * 2 * range_flow_coefficient - 0.5 * range_stage_loading) / 2
    return reaction

def calc_exit_axial_velocity(angle, range_exit_velocity):
    c_a_5 = range_exit_velocity * (numpy.cos(numpy.radians(range_alpha_3)))
    return c_a_5


data = []

for i in range_alpha_3:
    for j in stage_loading_we_get:
        for k in range_flow_coefficient:
                if 0.7 <= k <= 0.9: 
                    result = calc_reaction(i, j, k)
                    velocity = calc_exit_velocity(i)
                    if 0.35 <= result <= 0.65 and 0.3 <= velocity <= 0.55:
                        data.append((i, j, k, result, velocity))
            
df = pd.DataFrame(data, columns=['alpha_3', 'stage_loading', 'flow_coefficient', 'reaction', 'mach_number'])
df.to_csv('_outputs/output.csv', index=False)
